#### ```Import Dependencies```

In [22]:
!pip install 'stable-baselines3[extra]'

Defaulting to user installation because normal site-packages is not writeable


In [42]:
import os 
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

#### ```Load Environment```

In [24]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name, render_mode='human')

In [25]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, _,  info = env.step(action)
        score += reward
    print(f"Episodes: {episode} Score: {score}")
env.close()

/home/soumitra/.local/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episodes: 0 Score: 27.0
Episodes: 1 Score: 31.0
Episodes: 2 Score: 24.0
Episodes: 3 Score: 23.0
Episodes: 4 Score: 23.0


In [26]:
env.step??

Signature: env.step(action)
Source:   
    def step(self, action):
        """Steps through the environment and if the number of steps elapsed exceeds ``max_episode_steps`` then truncate.

        Args:
            action: The environment step action

        Returns:
            The environment step ``(observation, reward, terminated, truncated, info)`` with `truncated=True`
            if the number of steps elapsed >= max episode steps

        """
        observation, reward, terminated, truncated, info = self.env.step(action)
        self._elapsed_steps += 1

        if self._elapsed_steps >= self._max_episode_steps:
            truncated = True

        return observation, reward, terminated, truncated, info
File:      ~/.local/lib/python3.12/site-packages/gym/wrappers/time_limit.py
Type:      method

#### ```Understanding The Environment```

In [27]:
env.action_space.sample()

0

In [28]:
env.observation_space.sample()

array([-4.5758061e+00,  1.1205703e+38,  7.5159818e-02, -1.5981989e+38],
      dtype=float32)

#### ```Train an RL model```

In [43]:
environment_name = 'CartPole-v1'
env = Monitor(gym.make(environment_name, render_mode=None))
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, device='cpu')

AssertionError: 

In [44]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 856  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 584          |
|    iterations           | 2            |
|    time_elapsed         | 7            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0068477746 |
|    clip_fraction        | 0.0402       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.543       |
|    explained_variance   | 0.879        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0429       |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00406     |
|    value_loss           | 9.4          |
------------------------------------------
----------------

#### ```Save and Reload Model```

In [31]:
ppo_path = os.path.join('training', 'saved_models', 'ppo_model')

In [32]:
model.save(ppo_path)

In [33]:
del model

In [34]:
model = PPO.load(ppo_path, env = env)

/home/soumitra/.local/lib/python3.12/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


#### ```Evaluate the Model```

In [45]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

/home/soumitra/.local/lib/python3.12/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 500.0 +/- 0.0


In [36]:
print(env.reset())

[[ 0.03699933  0.03939426 -0.01058249  0.03432285]]


In [41]:
# Test the Trained Model with Rendering
test_env = gym.make(environment_name, render_mode='human')
obs, _ = test_env.reset()
done = False
score = 0
while not done:
    test_env.render()
    action, _ = model.predict(obs)
    obs, reward, done, _, info = test_env.step(action)
    score += reward
print(f"Score: {score}")

Score: 529.0


In [40]:
test_env.close()